In [28]:
import torch
import transformers
from transformers import AutoTokenizer, T5ForConditionalGeneration
import re

In [29]:
# Load the T5-small tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

In [47]:
def generate_subtasks(task_description, model, tokenizer):
    # Optimized prompt for T5-small
    prompt = f"Organize a list of actionable steps to accomplish the task: {task_description}\n"

    # Tokenize the input prompt
    inputs = tokenizer(prompt, return_tensors="pt", padding=True, truncation=True)

    # Generate output from the model
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            num_return_sequences=1,
            num_beams=3,            # Beam search for more coherent outputs
            max_length=200,         # Limit the response length
            early_stopping=True
        )

    # Decode the generated output
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Check if the model generated a response starting with a numbered list
    subtasks = []
    if "1." in generated_text:
        # Split by newline and process only lines with numbers and actions
        for line in generated_text.split("\n"):
            if line.strip().startswith(str(len(subtasks) + 1) + "."):
                subtasks.append(line.strip())
    else:
        # If the output doesn't start with a numbered list, handle it as raw text and format it
        subtasks = generated_text.split(". ")
        subtasks = [subtask.strip() for subtask in subtasks if subtask]

    return subtasks

In [50]:
task = "plan wedding"
subtasks = generate_subtasks(task, model, tokenizer)

In [51]:
for i, subtask in enumerate(subtasks, 1):
    print(f"{i}. {subtask}")

1. Organise a list of actionable steps to accomplish the task: plan wedding
